In [ ]:
import os
import tempfile
import time
from ansys.aedt.core import Hfss
import matplotlib.pyplot as plt
import numpy as np
#from Analise_CP import Z_par_g
#from Sintese_CP import synth_ant, synth_ant_pos
import pandas as pd
from scipy.optimize import minimize
from datetime import datetime
from scipy.odr import ODR, Model, RealData

AEDT_VERSION = "2024.2"
NUM_CORES = 4
NG_MODE = False
ConvP = 2

temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

project_name = os.path.join(temp_folder.name, "Spherical_Array.aedt")
design_name = "CP"
hfss = Hfss(version=AEDT_VERSION,
            non_graphical=NG_MODE,
            project=project_name,
            design = design_name,
            new_desktop=True,
            solution_type="Modal",
            )

In [ ]:
# Variables
hfss["a"] = "100mm"
a = 0.1
hfss["numseg"] = "19"
hfss["h"] = "1.524mm"
h=0.001524

hfss["Dthetaa"] = "33.212deg"
hfss["Dphia"] = "44.485deg"
hfss["phip1"] = "90deg"
hfss["thetap1"] = "96.357deg"

hfss["Rteflon"] = "2.05mm"
Rteflon= 0.00205
hfss["rprobe"] = "0.65mm"
rprobe = 0.00065
hfss["Hprobe"] = "15.07mm"
Hprobe = 0.01507
orange = [255,128,64]
red = [255,0,0]
blue = [0,255,255]

Alphas = np.array([35,35,-35,-35])
Betas = np.array([25,-25,25,-25])
Gammas = np.array([0,0,0,0])

npoints = 31

In [ ]:
from pyaedt.modeler.cad.polylines import PolylineSegment

try:
    for i in range(4):
        hfss.modeler.coordinate_systems[0].delete()
    hfss.modeler.set_working_coordinate_system("Global")
except:
    pass
# Generate Geometry
start_point = ["0mm", "0mm", "a"]
center_point = ["0mm", "0mm", "0mm"]

Gnd_Sphere_Name = "Gnd_Sphere"
Gnd_Sphere = hfss.modeler.create_polyline(
    points=[start_point, start_point],
    segment_type=[PolylineSegment(
        segment_type="AngularArc",
        arc_center=center_point,
        arc_angle="-180deg",
        arc_plane="ZX",
        num_seg="numseg"
    ), PolylineSegment(
        segment_type="Line"
    )],
    name=Gnd_Sphere_Name
)

hfss.modeler.cover_lines(Gnd_Sphere_Name)

hfss.modeler.sweep_around_axis(Gnd_Sphere_Name, "360deg", number_of_segments="2*numseg")

Gnd_Sphere.material_name = "copper"
Gnd_Sphere.color = orange

start_point = ["0mm", "0mm", "a+h"]

Diel_Sphere_Name = "Diel_Sphere"
Diel_Sphere = hfss.modeler.create_polyline(
    points=[start_point, start_point],
    segment_type=[PolylineSegment(
        segment_type="AngularArc",
        arc_center=center_point,
        arc_angle="-180deg",
        arc_plane="ZX",
        num_seg="numseg"
    ), PolylineSegment(
        segment_type="Line"
    )],
    name=Diel_Sphere_Name
)

hfss.modeler.cover_lines(Diel_Sphere_Name)

hfss.modeler.sweep_around_axis(Diel_Sphere_Name, "360deg", number_of_segments="2*numseg")

Diel_Sphere.material_name = "Arlon AD255A (tm)"

hfss.modeler.subtract(Diel_Sphere_Name, Gnd_Sphere_Name, keep_originals=True)

hfss.modeler.fit_all()

Probe_Sphere_Name = "Patch_Sphere"
Probe_Sphere = hfss.modeler.create_polyline(
    points=[["0mm", "0mm", "a+3*h"], ["0mm", "0mm", "a+3*h"]],
    segment_type=[PolylineSegment(
        segment_type="AngularArc",
        arc_center=center_point,
        arc_angle="-180deg",
        arc_plane="ZX",
        num_seg="numseg"
    ), PolylineSegment(
        segment_type="Line"
    )],
    name=Probe_Sphere_Name
)

hfss.modeler.cover_lines(Probe_Sphere_Name)

hfss.modeler.sweep_around_axis(Probe_Sphere_Name, "360deg", number_of_segments="2*numseg")

hfss.modeler.subtract(Probe_Sphere_Name, [Gnd_Sphere_Name, Diel_Sphere_Name], keep_originals=True)

## Ciclo for
for i in range(1,len(Alphas)+1):
    Patch_Name = "Patch" + str(i)
    Patch = hfss.modeler.create_polyline(
        points=start_point,
        segment_type=PolylineSegment(
            segment_type="AngularArc",
            arc_center=center_point,
            arc_angle="-180deg",
            arc_plane="ZX",
            num_seg="numseg"
        ),
        name=Patch_Name
    )

    hfss.modeler.sweep_around_axis(Patch_Name, "360deg", number_of_segments="2*numseg")
    
    hfss["Alpha"+str(i)] = str(Alphas[i-1])+"deg"
    hfss["Beta"+str(i)] = str(Betas[i-1])+"deg"
    hfss["Gamma"+str(i)] = str(Gammas[i-1])+"deg"

    coord = hfss.modeler.create_coordinate_system(mode = "zxz", origin=center_point, reference_cs="Global",name=Patch_Name+"_CS", phi = "Alpha" + str(i), theta = "Beta"+str(i))

    Cylinder_Name = "Cylinder"+str(i)
    Cylinder = hfss.modeler.create_polyline(
        points=[["0mm","0mm","a+2*h"],["0mm","a+2*h","a+2*h"],["0mm","a+2*h","-a-2*h"],["0mm","0mm","-a-2*h"],["0mm","0mm","a+2*h"]],
        segment_type=[PolylineSegment(
            segment_type="Line",
        ), PolylineSegment(
            segment_type="Line"
        ), PolylineSegment(
            segment_type="Line"
        ), PolylineSegment(
            segment_type="Line"
        )],
        name=Cylinder_Name
    )
    hfss.modeler.cover_lines(Cylinder_Name)
    hfss.modeler.rotate(Cylinder, "Z", angle="-Dphia/2")
    hfss.modeler.sweep_around_axis(Cylinder_Name, axis="Z", sweep_angle="Dphia-360deg")
    Cylinder.rotate("Y", "Gamma"+str(i))

    Cone_Upper_Name = "Cone_Up_" + str(i)
    Cone_Upper = hfss.modeler.create_cone(orientation = "Z", origin = ["0mm","0mm","a+2*h"],bottom_radius = "(a+2*h)*tan(90deg-Dthetaa/2)", top_radius=0, height = "-(a+2*h)", name = Cone_Upper_Name)
    Cone_Upper.rotate("Y", "Gamma"+str(i))

    Cone_Lower_Name = "Cone_Down_" + str(i)
    Cone_Lower = hfss.modeler.create_cone(orientation = "Z", origin = ["0mm","0mm","-(a+2*h)"],bottom_radius = "(a+2*h)*tan(90deg-Dthetaa/2)", top_radius=0, height = "a+2*h", name = Cone_Lower_Name)
    Cone_Lower.rotate("Y", "Gamma"+str(i))

    hfss.modeler.subtract(Patch_Name, [Cylinder_Name, Cone_Lower_Name, Cone_Upper_Name], keep_originals=False)
    hfss.assign_finite_conductivity(Patch_Name)

    Probe_Name = "Probe"+str(i)
    Probe = hfss.modeler.create_cylinder(orientation="Z", origin=["0mm","0mm","a+h"], radius="rprobe", height="-Hprobe", name = Probe_Name, material="copper")

    Teflon_Name = "Teflon"+str(i)
    Teflon = hfss.modeler.create_cylinder(orientation="Z", origin=["0mm","0mm","a+h/5"], radius="Rteflon", height="4*h/5-Hprobe", name = Teflon_Name, material="Teflon (tm)")

    hfss.modeler.subtract(Teflon_Name, Probe_Name, keep_originals=True)

    hfss.wave_port(assignment=hfss.modeler[Teflon_Name].faces[0], name = "Wave_Port"+str(i), deembed="Hprobe-h")

    hfss.modeler.rotate([Probe_Name, Teflon_Name], "Y", angle="thetap1")
    hfss.modeler.rotate([Probe_Name, Teflon_Name], "Z", angle="phip1")
    hfss.modeler.rotate([Probe_Name, Teflon_Name], "Y", angle="Gamma"+str(i))
    hfss.modeler.subtract(Gnd_Sphere_Name,[Teflon_Name, Probe_Name], keep_originals=True)
    hfss.modeler.subtract(Teflon_Name, Diel_Sphere_Name, keep_originals=True)
    hfss.modeler.subtract(Diel_Sphere_Name, Probe_Name, keep_originals=True)
    hfss.modeler.subtract(Probe_Name, Probe_Sphere_Name, keep_originals=True)

    ###

    hfss.modeler.set_working_coordinate_system("Global")

    hfss.modeler[Patch_Name].color = orange

hfss.modeler.delete(Probe_Sphere_Name)
for obj in hfss.modeler.object_names:
    hfss.modeler[obj].transparency = 0

In [ ]:

hfss.post.export_model_picture(full_name="C:\\Users\\ITA\\Documents\\Alunos\\JoaoPedroFalcao\\Mestrado\\Codes\\Arrays\\Array_View.png", show_axis=True, show_grid=True, show_ruler=True, show_region='Default', selections=["Diel_Sphere", "Patch1"], field_selections="all", orientation='back', width=0, height=0)

freq_range = [1.50042e9, 1.65042e9]      # Frequency range for analysis and post-processing.
center_freq = 1575.42e6            # Center frequency

open_box = hfss.create_open_region(frequency=str(center_freq/1e9)+"GHz", boundary = "PML")

setup = hfss.create_setup(name="GPS_Band", Frequency = str(center_freq/1e9)+"GHz", MaximumPasses=20, MinimumPasses=2, MinimumConvergedPasses=ConvP, PercentRefinement=30)

int_sweep = setup.add_sweep(name="GPS_Discrete", sweep_type="Discrete",
                             RangeStart=str(freq_range[0]/1e9)+"GHz", RangeEnd=str(freq_range[1]/1e9)+"GHz", RangeCount = npoints, SaveFields=True)

hfss.post.update_report_dynamically = True

# exp_cache = setup.enable_expression_cache(report_type="Far Fields", expressions="dB(AxialRatioValue)" ,intrinsics="Theta=\'90deg\' Phi=\'90deg\'", isrelativeconvergence= False, conv_criteria='0.2')

#converge = hfss.post.available_report_quantities(report_category="Terminal Solution Data",quantities_category="Expression Converge")
#hfss.post.create_report([converge[0], converge[1]], setup_sweep_name="GPS_Band : AdaptivePass", variations=hfss.get_nominal_variation(),primary_sweep_variable="Pass",report_category="Terminal Solution Data",plot_type="Rectangular Plot", plot_name="Convergence")

sweep = [freq_range[0], freq_range[1], npoints]
freqs = np.linspace(freq_range[0], freq_range[1], npoints)
rtd = 180/np.pi
Z0 = 50
iT = 0

setup.analyze()

reportZ = hfss.post.create_report(expressions=["re(ActiveZ(Wave_Port1:1))","im(ActiveZ(Wave_Port1:1))"], setup_sweep_name="GPS_Discrete", plot_name="Z_11_HFSS", variations = hfss.get_nominal_variation())
hfss.post.export_report_to_csv(project_dir="C:/Users/ITA/Documents/Alunos/JoaoPedroFalcao/Mestrado/Codes/Arrays",plot_name="Z_11_HFSS")
df = pd.read_csv('Z_11_HFSS.csv')
real_z = df["re(ActiveZ(Wave_Port1:1)) []"].to_numpy()
imag_z = df["im(ActiveZ(Wave_Port1:1)) []"].to_numpy()

In [ ]:
mesh_plot = hfss.post.create_fieldplot_volume(["Diel_Sphere"], quantity="Mesh", field_type="Fields", setup="GPS_Band : LastAdaptive", plot_name="Diel_Mesh")

Rad_3D = hfss.field_setups[0]
print(Rad_3D.name)
Rad_3D.theta_step = "1deg"
Rad_3D.phi_step = "1deg"
Rad_3D.theta_start = "-180deg"

variations = hfss.available_variations.nominal_values
variations["Freq"] = ["All"]
variations["Theta"] = ["90deg"]
variations["Phi"] = ["90deg"]
axialratio_ffd_plot = hfss.post.create_report(expressions="dB(AxialRatioValue)",
                                             setup_sweep_name="GPS_Discrete",
                                             variations=variations,
                                             primary_sweep_variable="Freq",
                                             context="3D",                     # Far-field setup is pre-defined.
                                             report_category="Far Fields",
                                             plot_type="Rectangular Plot",
                                             plot_name="Axial Ratio (dB)"
                                            )
hfss.post.export_report_to_csv(project_dir="C:/Users/ITA/Documents/Alunos/JoaoPedroFalcao/Mestrado/Codes/Arrays",plot_name="Axial Ratio (dB)")

In [ ]:
hfss.post.update_report_dynamically = True

variations = hfss.available_variations.nominal_values
variations["Freq"] = ["1.57542GHz"]
variations["Theta"] = ["All"]
variations["Phi"] = ["90deg"]
gain_ffd_plot = hfss.post.create_report(expressions=["dB(GainLHCP)", "dB(GainRHCP)"],
                                             setup_sweep_name="GPS_Discrete",
                                             variations=variations,
                                             primary_sweep_variable="Theta",
                                             context="3D",                     # Far-field setup is pre-defined.
                                             report_category="Far Fields",
                                             plot_type="Radiation Pattern",
                                             plot_name="Gain Phi90 (dB)"
                                            )

reportS = hfss.post.create_report(expressions=["dB(S(Wave_Port1,Wave_Port1))"], setup_sweep_name="GPS_Discrete", plot_name="S_11", variations = hfss.get_nominal_variation())

hfss.post.export_model_picture(full_name="C:\\Users\\ITA\\Documents\\Alunos\\JoaoPedroFalcao\\Mestrado\\Codes\\Arrays\\Back_View.png", show_axis=True, show_grid=True, show_ruler=True, show_region='Default', selections=["Diel_Sphere", "Patch1"], field_selections="all", orientation='back', width=0, height=0)

In [ ]:
variations = hfss.available_variations.nominal_values
variations["Freq"] = ["1.57542GHz"]
variations["Theta"] = ["All"]
variations["Phi"] = ["All"]
gain_ffd_plot = hfss.post.create_report(expressions=["re(rEPhi)"],
                                             setup_sweep_name="GPS_Discrete",
                                             variations=variations,
                                             context="3D",                     # Far-field setup is pre-defined.
                                             report_category="Far Fields",
                                             plot_type="3D Polar Plot",
                                             plot_name="rE_Phi_re"
                                            )
hfss.post.export_report_to_csv(project_dir="C:/Users/ITA/Documents/Alunos/JoaoPedroFalcao/Mestrado/Codes/Arrays",plot_name="rE_Phi_re")

gain_ffd_plot = hfss.post.create_report(expressions=["im(rEPhi)"],
                                             setup_sweep_name="GPS_Discrete",
                                             variations=variations,
                                             context="3D",                     # Far-field setup is pre-defined.
                                             report_category="Far Fields",
                                             plot_type="3D Polar Plot",
                                             plot_name="rE_Phi_im"
                                            )
hfss.post.export_report_to_csv(project_dir="C:/Users/ITA/Documents/Alunos/JoaoPedroFalcao/Mestrado/Codes/Arrays",plot_name="rE_Phi_im")

gain_ffd_plot = hfss.post.create_report(expressions=["re(rETheta)"],
                                             setup_sweep_name="GPS_Discrete",
                                             variations=variations,
                                             context="3D",                     # Far-field setup is pre-defined.
                                             report_category="Far Fields",
                                             plot_type="3D Polar Plot",
                                             plot_name="rE_Theta_re"
                                            )
hfss.post.export_report_to_csv(project_dir="C:/Users/ITA/Documents/Alunos/JoaoPedroFalcao/Mestrado/Codes/Arrays",plot_name="rE_Theta_re")

gain_ffd_plot = hfss.post.create_report(expressions=["im(rETheta)"],
                                             setup_sweep_name="GPS_Discrete",
                                             variations=variations,
                                             context="3D",                     # Far-field setup is pre-defined.
                                             report_category="Far Fields",
                                             plot_type="3D Polar Plot",
                                             plot_name="rE_Theta_im"
                                            )
hfss.post.export_report_to_csv(project_dir="C:/Users/ITA/Documents/Alunos/JoaoPedroFalcao/Mestrado/Codes/Arrays",plot_name="rE_Theta_im")

# Workbench

In [ ]:
# Recolher ganho total
# edit sources
# Plotar ganho nos planos principais

In [ ]:
hfss.save_project()
hfss.release_desktop()

# Wait 2 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(2)

temp_folder.cleanup()